In [13]:
DRIVER_PATH='C:/Users/gautier/CLionProjects/MLP_Rattrapage/Datasets/chromedriver.exe'
search_term = ["belgium flag"]
number_images = 100
target_path = './Test'


In [14]:
from selenium import webdriver

In [15]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [16]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")
        

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [17]:
def search_and_download(search_term:str,driver_path:str,target_path='./Test',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [18]:
import os 
import time
import requests
import io
from PIL import Image
import hashlib

for ele in search_term:
    search_and_download(search_term = ele, driver_path=DRIVER_PATH, target_path=target_path, number_images=number_images)

Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
SUCCESS - saved https://fvmstatic.s3.amazonaws.com/maps/m/BE-EPS-02-6001.png - as ./Test\belgium_flag\37b8d37524.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcST-ftxPX4qqHuA5xGj1Mo9KezlKm6aggm5AA&usqp=CAU - as ./Test\belgium_flag\803ebd7bbe.jpg
SUCCESS - saved https://cdn.w600.comps.canstockphoto.com/belgium-flag-on-button-eps-vector_csp64823540.jpg - as ./Test\belgium_flag\7200230a86.jpg
SUCCESS - saved https://images.crocs.com/is/image/Crocs/10006870_001_IS?&fmt=jpeg&qlt=85,1&op_sharpen=0&resMode=sharp2&op_usm=1,1,6,0&iccEmbed=0&printRes=72&wid=702&hei=586 - as ./Test\belgium_flag\646f4a1bb4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTyS8oWblWfyjgnmk8qr7SprX4wFQxQ2AMbGA&usqp=CAU - as ./Test\belgium_flag\1d882b802e.jpg
SUCCESS - saved https://as-df.fr/37085-large_default/clawgear-belgium-flag-patch-color.jpg - as ./Test\belgium_flag\c9711acadc

SUCCESS - saved https://riegerflags.com/PICTURE/1054-81-belgium-flag.jpg - as ./Test\belgium_flag\e8a39d02fb.jpg
SUCCESS - saved https://thumbs.dreamstime.com/b/flag-belgium-flag-belgium-colors-were-taken-coat-arms-duchy-brabant-vertical-design-may-be-131896983.jpg - as ./Test\belgium_flag\fa905c35af.jpg
SUCCESS - saved https://previews.123rf.com/images/captainvector/captainvector1509/captainvector150907658/45404780-belgium-flag-banni%C3%A8re.jpg - as ./Test\belgium_flag\ba441b123d.jpg
SUCCESS - saved https://47.cdn.ekm.net/ekmps/shops/0ec9a8/images/belgium-giant-flag-8ft-x-5ft-1913-p.gif?v=1 - as ./Test\belgium_flag\040b84b0b4.jpg
SUCCESS - saved https://comps.canstockphoto.fr/belgique-flag-drapeau-belgium-clipart-vecteur_csp49546302.jpg - as ./Test\belgium_flag\1edc0b3484.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTyMIeMdDVqsahagECC9tCjGxvfkA2yZSJmdQ&usqp=CAU - as ./Test\belgium_flag\eff8ef995f.jpg
SUCCESS - saved https://ak.picdn.net/shutterstock/video